In [4]:
# Import tensorflow library and mnist data from tensorflow

In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

We are going to define a function to initialize our weights 

In [8]:
def init_weights(shape, name):
    return tf.Variable(tf.random_normal(shape=shape,stddev=0.1, name=name))

# Defining the model construction

In [26]:
def model(X,w_h, w_h2, w_o, p_keep_input, p_keep_hidden):
    with tf.name_scope("layer_1"):
        X = tf.nn.dropout(X,p_keep_input)
        h = tf.matmul(X,w_h)
    with tf.name_scope("layer_2"):
        h = tf.nn.dropout(h, p_keep_hidden)
        h2 = tf.matmul(h, w_h2)
    with tf.name_scope("layer_3"):
        h2 = tf.nn.dropout(h2, p_keep_hidden)
        return tf.matmul(h2,w_o)

# Step 1 - Get the input data

In [9]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Step 2: Create input and output place holder for data

In [17]:
X = tf.placeholder("float",shape=[None, 784], name='X')
Y = tf.placeholder("float",shape=[None, 10], name='Y')

# Step 3: Initalize weights 

In [18]:
w_h = init_weights([784,625],'w_h')
w_h2 = init_weights([625, 625], 'w_h2')
w_o = init_weights([625, 10], 'w_o')

# Step 4 - Add histogram summaries for weights

In [19]:
tf.histogram_summary("w_h_summ", w_h)
tf.histogram_summary("w_h2_summ", w_h2)
tf.histogram_summary("w_o_summ", w_o)

<tf.Tensor 'HistogramSummary_2:0' shape=() dtype=string>

# Step 5 - Add dropout to input and hidden layers

In [20]:
p_keep_input = tf.placeholder("float", name="p_keep_input")
p_keep_hidden = tf.placeholder("float", name="p_keep_hidden")

# Step 6 - Create model

In [27]:
py_x = model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden)

# Step 7 - Create cost function

In [32]:
with tf.name_scope("cost"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x,Y))
    train_op = tf.train.RMSPropOptimizer(0.001,0.9).minimize(cost)
    # Add scalar summary for cost tensor
    tf.scalar_summary("cost", cost)

# Step 8 - Measure Accuracy

In [33]:
with tf.name_scope("Accuracy"):
    correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(py_x, 1)) # Count correct predictions
    acc_op = tf.reduce_mean(tf.cast(correct_pred, "float")) # Cast boolean to float to average
    # Add scalar summary for accuracy tensor
    tf.scalar_summary("Accuracy", acc_op)

# Step 9 Create a session

In [ ]:
with tf.Session() as sess:
    # Step 10 create a log writer. run 'tensorboard --logdir=./logs/nn_logs'
    writer = tf.train.SummaryWriter("./logs/nn_logs", sess.graph)
    merged = tf.merge_all_summaries()
    
    # Step 11 you need to initialize all variables
    tf.initialize_all_variables().run()
    
    #Step 12 train the  model
    for i in range(10):
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)):
            sess.run(train_op,feed_dict = {X: trX[start:end], Y: trY[start:end],
                                          p_keep_input: 0.8, p_keep_hidden: 0.5})
            summary, acc = sess.run([merged, acc_op], feed_dict={X: teX, Y: teY,
                                                                 p_keep_input: 1.0, p_keep_hidden: 1.0})
        writer.add_summary(summary, i)
        print(i,acc)

In [14]:
tf.scalar_summary?